Courses file uploaded

In [ ]:
file_path = r"Formations_Cours.csv"
courses_df = pd.read_csv(file_path)
courses_df.head()

Coursera site scrapping

# Set up the Chrome driver
path = '/Users/yhema/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

# Load the file with domain, sous-domain, and course details
file_path = r"E:\M2\BDA\SEFORMER_FORMATIONS_PAS_DE_FORMATEURS.csv"
data = pd.read_csv(file_path)

# Prepare the CSV file for writing results
output_file = "course_links_with_titles_cont.csv"

with open(output_file, mode="w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Domain", "Sous-Domain", "Course Name", "Title 1", "Link 1", "Title 2", "Link 2", "Title 3", "Link 3"])

    # Loop through each row to build the Coursera URL and extract course titles and links (up to 5 rows)
    for _ , row in data.iterrows():
        
        domain = row['DOMAINE_CATGEGORIE']
        sous_domain = row['SOUS_DOMAINE_CATEGORIE']
        course_name = row['FORMATION_COURS']

        # Construct Coursera search URL
        coursera_search_url = f"https://www.coursera.org/search?query={domain}+{sous_domain}+{course_name}"
        driver.get(coursera_search_url)

        # Accept cookies if needed (optional)
        try:
            accept_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Accept")]'))
            )
            accept_button.click()
        except Exception:
            pass

        # Extract the top 3 course titles and links
        titles_links = []
        try:
            course_elements = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, '//a[contains(@class,"cds-CommonCard-titleLink")]'))
            )

            for i, course_element in enumerate(course_elements[:3]):
                title = course_element.text
                link = course_element.get_attribute("href")
                titles_links.append((title, link))

            # Fill missing entries if less than 3 courses are found
            while len(titles_links) < 3:
                titles_links.append(("Not found", "Not found"))

            # Write the row with domain, sous-domain, course name, and the 3 titles and links
            row_data = [domain, sous_domain, course_name]
            for title, link in titles_links:
                row_data.extend([title, link])

            writer.writerow(row_data)

        except Exception as e:
            print(f"Error extracting data for '{domain} - {sous_domain} - {course_name}': {e}")

print(f"Course links and titles saved to {output_file}")

# Close the driver
driver.quit()
